In [19]:
import torch
from torch.nn import functional as F
from torch import nn
import math
import random
import numpy as np
import requests

Work on the bigram model first

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = open('text.txt').read()
vocab = list(sorted(set(data)))
vocab_size = len(vocab)

print(f'Vocab: {vocab[:10]}.')
print(f'Vocab size: {vocab_size}')

stoi = {c:i for i, c in enumerate(vocab)}
itos = {i:c for i, c in enumerate(vocab)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join([itos[i] for i in i])

print(decode(encode(data[1100:1150])))
data = torch.tensor(encode(data))

Vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3'].
Vocab size: 65
very dog to the commonalty.

Second Citizen:
Consi


In [21]:
block_size = 8
batch_size = 32

n_tr = int(len(data) * 0.9)
n_val = len(data) - n_tr

train = data[:n_tr]
val = data[n_tr+1:]

print(f'Training samples: {train.shape}')
print(f'Validation samples: {val.shape}')

def make_batches(split):

    data = train if split == 'train' else val
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    
    return x, y

Training samples: torch.Size([1003854])
Validation samples: torch.Size([111539])


In [22]:
Xb, Yb = make_batches('train')
Xb.shape

torch.Size([32, 8])

In [ ]:
# read the data
# create vocab
# do vocab size
# create training and validation splits
# create data loader
# create the bi-gram lookup table

In [48]:
Xb.shape

torch.Size([32, 8])

In [49]:
emb_table = nn.Embedding(vocab_size, vocab_size).to(device) # (65, 65)
logits = emb_table(Xb) # (32, 8, 65)
print(logits.shape)
B, T, C = logits.shape
logits = logits.view(B*T, C)
targets = Yb.view(B*T)
loss = F.cross_entropy(logits, targets)

torch.Size([32, 8, 65])


In [67]:
Yb.shape

torch.Size([32, 8])

In [69]:
logits[0], targets[0]

(tensor([ 0.0498,  0.3900,  0.8843, -0.6404, -0.5388, -1.1646, -0.5890, -0.0852,
         -0.4918,  0.3321, -0.0126, -0.5153, -0.4854, -0.6589, -0.6387, -0.3921,
          2.1584, -0.7153,  0.2292, -1.4770,  1.6679,  0.7794, -0.0394,  0.7323,
          0.3798,  0.5197,  0.2711,  0.1370,  0.8722,  1.5015, -1.0303,  1.2152,
         -0.3852, -0.2712, -1.2364,  1.2883, -1.7084, -1.1360,  0.0991,  0.3909,
          1.1893, -0.3269,  0.0831,  0.6745, -0.4875,  0.9542, -0.7550, -1.1399,
         -0.7167, -0.1602,  0.0048, -0.1801, -0.3668,  1.8157,  1.1692, -0.3340,
          0.9738,  0.1174,  0.0178, -0.4693, -0.9466, -1.0655,  1.6465, -0.2454,
          3.0582], device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(57, device='cuda:0'))

In [46]:
targets.shape

torch.Size([256])

In [47]:
targets[0]

tensor(57, device='cuda:0')

In [70]:
import torch
import torch.nn.functional as F

# Define logits and target
logits_i = torch.tensor([ 0.0498,  0.3900,  0.8843, -0.6404, -0.5388, -1.1646, -0.5890, -0.0852,
         -0.4918,  0.3321, -0.0126, -0.5153, -0.4854, -0.6589, -0.6387, -0.3921,
          2.1584, -0.7153,  0.2292, -1.4770,  1.6679,  0.7794, -0.0394,  0.7323,
          0.3798,  0.5197,  0.2711,  0.1370,  0.8722,  1.5015, -1.0303,  1.2152,
         -0.3852, -0.2712, -1.2364,  1.2883, -1.7084, -1.1360,  0.0991,  0.3909,
          1.1893, -0.3269,  0.0831,  0.6745, -0.4875,  0.9542, -0.7550, -1.1399,
         -0.7167, -0.1602,  0.0048, -0.1801, -0.3668,  1.8157,  1.1692, -0.3340,
          0.9738,  0.1174,  0.0178, -0.4693, -0.9466, -1.0655,  1.6465, -0.2454,
          3.0582], requires_grad=True)

target_i = torch.tensor(57)

# Compute cross entropy loss
loss_i = F.cross_entropy(logits_i.unsqueeze(0), target_i.unsqueeze(0))
loss_i

tensor(4.6533, grad_fn=<NllLossBackward0>)

In [72]:
import torch

# Define logits and target
logits_i = torch.tensor([ 0.0498,  0.3900,  0.8843, -0.6404, -0.5388, -1.1646, -0.5890, -0.0852,
         -0.4918,  0.3321, -0.0126, -0.5153, -0.4854, -0.6589, -0.6387, -0.3921,
          2.1584, -0.7153,  0.2292, -1.4770,  1.6679,  0.7794, -0.0394,  0.7323,
          0.3798,  0.5197,  0.2711,  0.1370,  0.8722,  1.5015, -1.0303,  1.2152,
         -0.3852, -0.2712, -1.2364,  1.2883, -1.7084, -1.1360,  0.0991,  0.3909,
          1.1893, -0.3269,  0.0831,  0.6745, -0.4875,  0.9542, -0.7550, -1.1399,
         -0.7167, -0.1602,  0.0048, -0.1801, -0.3668,  1.8157,  1.1692, -0.3340,
          0.9738,  0.1174,  0.0178, -0.4693, -0.9466, -1.0655,  1.6465, -0.2454,
          3.0582], requires_grad=True)

target_i = torch.tensor(57)

# Apply the softmax function to the logits to get probabilities
probabilities = torch.softmax(logits_i, dim=0)

# Compute the negative log likelihood of the true class
nll = -torch.log(probabilities[target_i])
nll

tensor(4.6533, grad_fn=<NegBackward0>)

In [73]:
probabilities

tensor([0.0089, 0.0125, 0.0205, 0.0045, 0.0049, 0.0026, 0.0047, 0.0078, 0.0052,
        0.0118, 0.0084, 0.0051, 0.0052, 0.0044, 0.0045, 0.0057, 0.0734, 0.0041,
        0.0107, 0.0019, 0.0449, 0.0185, 0.0081, 0.0176, 0.0124, 0.0143, 0.0111,
        0.0097, 0.0203, 0.0380, 0.0030, 0.0286, 0.0058, 0.0065, 0.0025, 0.0307,
        0.0015, 0.0027, 0.0094, 0.0125, 0.0278, 0.0061, 0.0092, 0.0166, 0.0052,
        0.0220, 0.0040, 0.0027, 0.0041, 0.0072, 0.0085, 0.0071, 0.0059, 0.0521,
        0.0273, 0.0061, 0.0224, 0.0095, 0.0086, 0.0053, 0.0033, 0.0029, 0.0440,
        0.0066, 0.1804], grad_fn=<SoftmaxBackward0>)

In [39]:
max(logits[0])

tensor(1.7083, device='cuda:0', grad_fn=<UnbindBackward0>)

In [38]:
logits[0]

tensor([-1.8095e+00, -1.1302e+00, -1.2551e-01,  1.4313e+00, -2.7304e+00,
         1.3788e+00,  1.0882e+00,  1.2311e+00, -7.3206e-01,  6.4607e-01,
        -4.5302e-01, -8.2092e-01, -9.0916e-01, -8.6847e-02,  1.3041e+00,
         1.0103e+00, -5.9111e-01, -1.6377e-01, -6.1103e-01,  3.8011e-01,
         1.5490e+00, -5.8434e-01,  4.7227e-01,  2.2436e-01,  9.9161e-01,
         9.4985e-01, -9.4863e-02, -1.6170e+00, -8.2019e-01, -6.4654e-01,
        -1.3319e-01,  7.8813e-03, -1.1176e+00,  6.4368e-01,  1.8809e-01,
         7.1894e-01,  1.7083e+00,  1.4248e+00,  1.2541e+00,  1.1685e+00,
         6.8397e-01, -4.5269e-01, -1.2468e+00, -8.8542e-01, -1.1985e-01,
         7.2042e-01, -3.5219e-01, -2.6869e+00,  1.2400e+00,  6.7754e-01,
         1.5006e-01,  4.5240e-01, -1.0221e+00,  6.9845e-01, -6.6867e-02,
        -1.3289e-01, -7.6452e-01,  1.3142e+00, -2.6511e-01, -3.3069e-01,
         6.2809e-01, -2.0188e-03,  6.8674e-01,  5.9731e-01, -7.0109e-01],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [26]:
loss = F.cross_entropy(target=y, input=x)

RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'

In [ ]:
class BigramLanguageModel(nn.Module):
    def __innit__(self, vocab_size):
        super().__innit__()

        self.embedding_table = nn.Embbeding(vocab_size, vocab_size)

    def forward(self, ix, targets=None):

        logits = self.embedding_table(ix)

        if targets == None:
            None
        else:
            loss = F.cross_entropy(target=targets, input=logits)
